In [1]:
from sqlalchemy import create_engine
import pandas as pd

db_user = 'postgres'
db_password = 'xxxx'
db_host = 'localhost'
db_port = '5432'
db_name = 'postgres'

connection_string = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
engine = create_engine(connection_string)

In [2]:
with engine.connect() as conn:
    query = "select * from netflix"
    df = pd.read_sql(query, conn)
df.head()

,show_id,type,title,director,actors,country,rating,duration,listed_in,description,date_added,release_year
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,Unknown,United States,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",2021-09-25,2020
1,s2,TV Show,Blood & Water,Unknown,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",2021-09-24,2021
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",Unknown,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,2021-09-24,2021
3,s4,TV Show,Jailbirds New Orleans,Unknown,Unknown,Unknown,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",2021-09-24,2021
4,s5,TV Show,Kota Factory,Unknown,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,2021-09-24,2021


### __#1: Top 15 Movie Genres on Netflix__
---

1. Splitting the genres from `listed_in` as multiple genres could be given
2. Counting the number of movies
3. Grouping by genre
4. Ordering my movie count and getting the top 15 genres

In [3]:
with engine.connect() as conn:
    query = """
    SELECT 
        TRIM(REGEXP_SPLIT_TO_TABLE(listed_in, ',')) AS genre,
        count(show_id) AS movies
    FROM netflix
    WHERE type = 'Movie'
    GROUP BY genre
    ORDER BY movies DESC
    LIMIT 15
    """
    df = pd.read_sql(query, conn)
df

,genre,movies
0,International Movies,2752
1,Dramas,2427
2,Comedies,1674
3,Documentaries,869
4,Action & Adventure,859
5,Independent Movies,756
6,Children & Family Movies,641
7,Romantic Movies,616
8,Thrillers,577
9,Music & Musicals,375


### __#2: Top 15 Longest TV Shows Not Specifically Made For Children__
---

1. Creating a new column named `seasons` from `duration` for the display of TV show length
2. Filtering for TV shows and not children-focused TV shows
3. Selecting the longers series' by ordering and limiting the results

In [4]:
with engine.connect() as conn:
    query = """
    SELECT 
        title,
        listed_in as genre,
        rating,
        CAST(SUBSTRING(duration FROM 1 FOR 2) AS INTEGER) AS seasons
    FROM netflix
    WHERE type = 'TV Show' and 
        rating NOT IN ('TV-Y','TV-Y7','TV-Y7-FV')
    ORDER BY seasons DESC
    LIMIT 15
    """
    df = pd.read_sql(query, conn)
df

,title,genre,rating,seasons
0,Grey's Anatomy,"Romantic TV Shows, TV Dramas",TV-14,17
1,NCIS,"Crime TV Shows, TV Dramas, TV Mysteries",TV-14,15
2,Supernatural,"Classic & Cult TV, TV Action & Adventure, TV H...",TV-14,15
3,Heartland,TV Dramas,TV-14,13
4,COMEDIANS of the world,"Stand-Up Comedy & Talk Shows, TV Comedies",TV-MA,13
5,Red vs. Blue,"TV Action & Adventure, TV Comedies, TV Sci-Fi ...",NR,13
6,Criminal Minds,"Crime TV Shows, TV Dramas, TV Mysteries",TV-14,12
7,Trailer Park Boys,"Classic & Cult TV, Crime TV Shows, Internation...",TV-MA,12
8,Frasier,"Classic & Cult TV, TV Comedies",TV-PG,11
9,Cheers,"Classic & Cult TV, TV Comedies",TV-PG,11


### __#3: Top 10 American or British Movie Actors and their Number of Available Movies per Top Genres on Netflix__
---

1. *Subquery creation*
    1. Splitting the cast on commas for individual actor analysis
    2. Applying `CASE` statement combined with `SUM` and `WHEN` to get the number of movies for each genre
    3. Filtering for UK and US actors and actresses, and movies
    4. Grouping the results by actors
2. Selecting the necessary fields and creating the `combined_genres` column

In [5]:
with engine.connect() as conn:
    query = """
    SELECT
        actor, 
        dramas, 
        comedies, 
        actions,
        drama_comedies+action_comedies+action_dramas as combined_genres,
        other_genres,
        movies

    FROM 
        (
        SELECT 
            TRIM(REGEXP_SPLIT_TO_TABLE(actors, ',')) AS actor,

            SUM(CASE WHEN listed_in LIKE '%%Dramas%%' AND
                            listed_in NOT LIKE '%%Comedies%%' AND
                            listed_in NOT LIKE '%%Action & Adventure%%'
                            then 1 else 0 end) AS dramas,

            SUM(CASE WHEN listed_in LIKE '%%Comedies%%' AND
                            listed_in NOT LIKE '%%Dramas%%' AND
                            listed_in NOT LIKE '%%Action & Adventure%%'
                            then 1 else 0 end) AS comedies,

            SUM(CASE WHEN listed_in LIKE '%%Action & Adventure%%' AND
                            listed_in NOT LIKE '%%Comedies%%' AND
                            listed_in NOT LIKE '%%Dramas%%' 
                            then 1 else 0 end) AS actions,

            SUM(CASE WHEN listed_in LIKE '%%Comedies%%' AND 
                            listed_in LIKE '%%Dramas%%' AND 
                            listed_in NOT LIKE '%%Action & Adventure%%' 
                            then 1 else 0 end) AS drama_comedies,

            SUM(CASE WHEN listed_in LIKE '%%Comedies%%' AND 
                            listed_in LIKE '%%Action & Adventure%%' AND 
                            listed_in NOT LIKE '%%Dramas%%' 
                            then 1 else 0 end) AS action_comedies,

            SUM(CASE WHEN listed_in LIKE '%%Action & Adventure%%' AND 
                            listed_in LIKE '%%Dramas%%' AND 
                            listed_in NOT LIKE '%%Comedies%%' 
                            then 1 else 0 end) AS action_dramas,
                            
            SUM(CASE WHEN listed_in NOT LIKE '%%Dramas%%' AND 
                            listed_in NOT LIKE '%%Comedies%%' AND 
                            listed_in NOT LIKE '%%Action & Adventure%%' 
                            then 1 else 0 end) AS other_genres,

            count(show_id) as movies
        FROM netflix
        WHERE actors != 'Unknown' AND 
                country LIKE ANY ('{%%United States%%,%%United Kingdom%%}') and
                type = 'Movie'
        GROUP BY actor
        ORDER BY movies DESC
        LIMIT 10
        )    
    """
    df = pd.read_sql(query, conn)
df

,actor,dramas,comedies,actions,combined_genres,other_genres,movies
0,Samuel L. Jackson,6,3,6,5,1,21
1,Adam Sandler,1,14,0,4,1,20
2,James Franco,7,2,3,6,1,19
3,Nicolas Cage,4,2,7,2,3,18
4,John Cleese,0,10,2,2,3,17
5,Morgan Freeman,4,1,4,2,5,16
6,Willem Dafoe,6,0,5,3,1,15
7,Bruce Willis,1,2,8,3,1,15
8,Tara Strong,0,9,1,0,5,15
9,Fred Tatasciore,0,4,3,1,7,15


### __#4: Months with the Highest Movies Additions to Netflix, yearly__
---

1. *Subquery 1* - `monthly_counts`
    1. Extracting the months and years separately from the `date_added` column
    2. Filtering for years between 2015 and 2022
    3. Grouping by the new `year` and `month` columns
2. *Subquery 2* - `yearly_max_counts`
    1. This subquery is in the `WHERE` clause
    2. The same query as the first one, only I can now refer to the `added_movies_shows` column and retrieve the highest number, yearly
3. Selecting the 3 column and ordering the results by `year`


In [6]:
with engine.connect() as conn:
    query = """
    SELECT 
        year, 
        month, 
        added_movies_shows
    FROM (
        SELECT 
            EXTRACT(YEAR FROM date_added) AS year, 
            EXTRACT(MONTH FROM date_added) AS month, 
            COUNT(show_id) AS added_movies_shows
        FROM netflix
        WHERE date_added BETWEEN '2015-01-01' AND '2022-01-01'
        GROUP BY year, month
    ) AS monthly_counts
    WHERE (year, added_movies_shows) IN (
        SELECT 
            year, 
            MAX(added_movies_shows) AS max_added_movies_shows
        FROM (
            SELECT 
                EXTRACT(YEAR FROM date_added) AS year, 
                EXTRACT(MONTH FROM date_added) AS month, 
                COUNT(show_id) AS added_movies_shows
            FROM netflix
            WHERE date_added BETWEEN '2015-01-01' AND '2022-01-01'
            GROUP BY year, month
        ) AS yearly_max_counts
        GROUP BY year
    )
    ORDER BY year
    """
    df = pd.read_sql(query, conn)
df

,year,month,added_movies_shows
0,2015.0,12.0,21
1,2016.0,12.0,96
2,2017.0,10.0,126
3,2018.0,10.0,191
4,2019.0,11.0,255
5,2020.0,1.0,205
6,2021.0,7.0,257


In [7]:
with engine.connect() as conn:
    query = """
    SELECT 
        DATE_TRUNC('MONTH', date_added)::date date,
        COUNT(show_id) AS added_movies_shows
    FROM netflix
    WHERE EXTRACT(YEAR FROM date_added) = 2021
    GROUP BY date
    """
    df = pd.read_sql(query, conn)
df

,date,added_movies_shows
0,2021-01-01,132
1,2021-02-01,109
2,2021-03-01,112
3,2021-04-01,188
4,2021-05-01,132
5,2021-06-01,207
6,2021-07-01,257
7,2021-08-01,178
8,2021-09-01,183


> As per the available data (before October 2021), Netflix tend to add more movies and shows in the colder months.

### __#5: Top 10 Countries by Number of Releases per Year, between 2019 and 2021__
---

1. *CTE 1* - `country_split`
    1. Splitting the countries because the database has multiple countries as one value a lot of times.
    2. Main where clause: filtering for releases after 2017
2. *CTE 2* - `shows_per_country`
    1. Calculating the number of shows for each country
3. *CTE 3* - `top_countries`
    1. Ordering the countries based on total releases (shows and movies)
    2. Only keeping the top 10 countries with the most releases
    3. Creating the country ranking
4. *CTE 4* - `releases_per_top_countries`
    1. Working from CTE1 - `country_split`
    2. Counting the distinct releases grouped by countries and release years
    3. Inner joining CTE3 - `top_countries` to only show the data for the top 10 countries based on summarized releases
    4. Grouping by countries and years too
5. *CTE 5* - `growth_calc`
    1. Creating one more CTE to be able to calculate 'growth' - the change in releases from the previous years and countries
6. *Final Query*
    1. Removing assisting rows used for 'growth' calculation

In [8]:
with engine.connect() as conn:
    query = """
    WITH country_split AS (
        SELECT 
            show_id,
            TRIM(REGEXP_SPLIT_TO_TABLE(country, ',')) AS show_country,
            release_year
        FROM netflix
        WHERE 
            release_year > 2017 
            and country != 'Unknown'
    ),
    shows_per_country AS (
        SELECT 
            show_country AS country,
            COUNT(DISTINCT show_id) AS total_releases
        FROM country_split
        GROUP BY country
    ),
    top_countries AS (
        SELECT 
            country, 
            DENSE_RANK() OVER (ORDER BY total_releases DESC) AS country_rank
        FROM shows_per_country
        ORDER BY total_releases DESC
        LIMIT 10
    ),
    releases_per_top_countries AS (
        SELECT 
            tc.country,
            cs.release_year,
            COUNT(DISTINCT cs.show_id) AS number_of_releases,
            country_rank
        FROM country_split cs
        INNER JOIN top_countries tc ON cs.show_country = tc.country
        GROUP BY 
            tc.country, 
            cs.release_year, 
            tc.country_rank
    ),
    growth_calc AS (
        SELECT 
            country,
            release_year,
            number_of_releases,
            number_of_releases - LAG(number_of_releases) OVER (PARTITION BY country ORDER BY release_year ASC) AS change_in_releases
        FROM releases_per_top_countries
        ORDER BY 
            country_rank, 
            release_year
    )
    SELECT * 
    FROM growth_calc
    WHERE change_in_releases != 0
    """
    df = pd.read_sql(query, conn)
df


,country,release_year,number_of_releases,change_in_releases
0,United States,2019,428,-29
1,United States,2020,384,-44
2,United States,2021,173,-211
3,India,2019,93,-8
4,India,2020,77,-16
5,India,2021,35,-42
6,United Kingdom,2019,68,-28
7,United Kingdom,2021,36,-32
8,Canada,2019,44,-23
9,Canada,2020,39,-5
